In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import datetime

c_demographics = pd.read_csv('customer_demographics.csv')
c_transaction = pd.read_csv('customer_transaction_data.csv')
item_data = pd.read_csv('item_data.csv')
coupon_item_map = pd.read_csv('coupon_item_mapping.csv')
campaignData = pd.read_csv('campaign_data.csv')
train   = pd.read_csv('train.csv')
test = pd.read_csv('test_QyjYwdj.csv')
sampleSubm = pd.read_csv('sample_submission_Byiv0dS.csv')

item_data['brand_type_category'] = item_data['brand_type'] + '_' + item_data['category']
c_transaction = pd.merge(c_transaction, item_data, on = 'item_id', how = 'left')

c_transaction['date'] = pd.to_datetime(c_transaction['date'], format= '%Y-%m-%d')
campaignData['start_date'] = pd.to_datetime(campaignData['start_date'], format= '%d/%m/%y')
campaignData['end_date'] = pd.to_datetime(campaignData['end_date'], format= '%d/%m/%y')

campaignData = campaignData.sort_values(by = 'start_date')
campaignData['duration'] = campaignData['end_date'] - campaignData['start_date']

c_transaction

c_transaction['total_disc'] = c_transaction['other_discount'] + c_transaction.coupon_discount
c_transaction['Total_Bill'] = c_transaction['selling_price'] * c_transaction['quantity'] + c_transaction['total_disc']
c_transaction['dayofweek'] = c_transaction['date'].dt.dayofweek
c_transaction['month'] = c_transaction['date'].dt.month
c_transaction['is_monthEnd'] = c_transaction['date'].dt.day > 25
c_transaction['is_monthStart'] = c_transaction['date'].dt.day < 7
c_transaction['disc_perc'] = abs(c_transaction['other_discount'] + c_transaction['coupon_discount']) / c_transaction['selling_price']
c_transaction['is_coupoun_disc'] = c_transaction['coupon_discount'] < 0
c_transaction['brand_category'] = c_transaction['brand'].astype('str') + '_' + c_transaction['category']
coupon_item_dict = {x : coupon_item_map[coupon_item_map.coupon_id == x]['item_id'].values for x in coupon_item_map.coupon_id.unique()}
c_transaction['dayofmonth'] = c_transaction['date'].dt.day

treat_age = {'70+' : 72, '46-55' : 50.5, '36-45' : 40.5, '26-35' : 30.5, '56-70' : 63, '18-25' : 21.5}
c_demographics['age_approximated'] = c_demographics['age_range'].apply(lambda x : treat_age[x])
c_demographics['income_bucket_div_age'] = c_demographics['income_bracket'] / c_demographics['age_approximated']
treat_family_size = {'1' : 1, '2' : 2, '3' : 3, '4' : 4, '5+' : 5}
c_demographics['family_size_'] = c_demographics['family_size'].map(treat_family_size)
c_demographics['income_div_family_size'] = c_demographics['income_bracket'] / c_demographics['family_size_']



c_demographics['no_of_children'] = c_demographics['no_of_children'].map({'1' : 1, '2' : 2, '3+' : 3})
c_demographics['income_bucket_div_no_of_child'] = c_demographics['income_bracket'] / c_demographics['no_of_children']

c_demographics['family_to_number_of_child'] = c_demographics['family_size_'] / c_demographics['no_of_children']
c_demographics.drop('family_size_', axis = 1, inplace = True)

campaignData['duration'] = campaignData['duration'].dt.days

## MAKE SURE FEAT ENG 2 has completed it's run.

train = pd.read_pickle('grpd_by_train.pkl')
test = pd.read_pickle('grpd_by_test.pkl')

from tqdm import tqdm

c_transaction.columns

def do_parallel_train(campIDs):
    Ans = []
    temp = train[train['campaign_id'] == campIDs]
    date = campaignData[campaignData.campaign_id == campIDs]['start_date'].values[0]
    tempTransaction = c_transaction[c_transaction.date < date]
    for custID in temp.customer_id.unique():
        print('{} ...... {}'.format(custID, campIDs))
        tempTransactionCust = tempTransaction[tempTransaction['customer_id'] == custID]
        coupnsTrain = train[(train.customer_id == custID) & (train.campaign_id == campIDs)]
        for coupID in coupnsTrain.coupon_id.unique():
            count = 0
            itemsForCoupon = coupon_item_dict[coupID]
            itemPastSellingMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].mean()
            itemPastSellingStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].std()
            itemPastSellingMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].max()
            itemPastSellingMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].min()
            itemPastOtherMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].mean()
            itemPastOtherStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].std()
            itemPastOtherMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].max()
            itemPastOtherMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].min()
            itemPastCouponMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].mean()
            itemPastCouponStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].std()
            itemPastCouponMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].max()
            itemPastCouponMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].min()
            itemPastTotal_DiscMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].mean()
            itemPastTotal_DiscStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].std()
            itemPastTotal_DiscMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].max()
            itemPastTotal_DiscMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].min()
            itemPastTotalBillMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].mean()
            itemPastTotalBillStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].std()
            itemPastTotalBillMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].max()
            itemPastTotalBillMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].min()
            itemPastDiscPercMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].mean()
            itemPastDiscPercStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].std()
            itemPastDiscPercMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].max()
            itemPastDiscPercMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].min()
            itemPastIsCouponDiscSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_coupoun_disc'].sum()
            itemPastIsCouponDiscMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_coupoun_disc'].mean()
            itemPastIsMonthEndSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthEnd'].sum()
            itemPastIsMonthEndMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthEnd'].mean()
            itemPastIsMonthStartSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthStart'].sum()
            itemPastIsMonthStartMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthStart'].mean()
            itemPastDayOfMonthMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['dayofmonth'].mean()
            for item in tempTransactionCust.item_id:
                if item in itemsForCoupon:
                    count+=1
            Ans.append((campIDs, custID, coupID, count, 
                        itemPastSellingMean ,
                        itemPastSellingStd ,
                        itemPastSellingMax ,
                        itemPastSellingMin ,
                        itemPastOtherMean ,
                        itemPastOtherStd ,
                        itemPastOtherMax, 
                        itemPastOtherMin , 
                        itemPastCouponMean ,
                        itemPastCouponStd , 
                        itemPastCouponMax , 
                        itemPastCouponMin , 
                        itemPastTotal_DiscMean ,
                        itemPastTotal_DiscStd , 
                        itemPastTotal_DiscMax , 
                        itemPastTotal_DiscMin , 
                        itemPastTotalBillMean , 
                        itemPastTotalBillStd ,
                        itemPastTotalBillMax , 
                        itemPastTotalBillMin , 
                        itemPastDiscPercMean , 
                        itemPastDiscPercStd , 
                        itemPastDiscPercMax , 
                        itemPastDiscPercMin , 
                        itemPastIsCouponDiscSum ,  
                        itemPastIsCouponDiscMean ,  
                        itemPastIsMonthEndSum , 
                        itemPastIsMonthEndMean ,
                        itemPastIsMonthStartSum ,  
                        itemPastIsMonthStartMean , 
                        itemPastDayOfMonthMean , 
                        ))

    return Ans

import multiprocessing

p = multiprocessing.Pool(processes=8)

Ans_train = p.map(do_parallel_train, train.campaign_id.unique())

Ans = []
for x in Ans_train:
    for i in x:
        Ans.append(i)

itemCount = pd.DataFrame(Ans, columns=['campaign_id', 'customer_id', 'coupon_id', 'prev_int_count',
                                       'itemPastSellingMean' ,
                                        'itemPastSellingStd' ,
                                        'itemPastSellingMax' ,
                                        'itemPastSellingMin' ,
                                        'itemPastOtherMean' ,
                                        'itemPastOtherStd' ,
                                        'itemPastOtherMax',
                                        'itemPastOtherMin' ,
                                        'itemPastCouponMean' ,
                                        'itemPastCouponStd' ,
                                        'itemPastCouponMax' ,
                                        'itemPastCouponMin' ,
                                        'itemPastTotal_DiscMean' ,
                                        'itemPastTotal_DiscStd' ,
                                        'itemPastTotal_DiscMax' ,
                                        'itemPastTotal_DiscMin' ,
                                        'itemPastTotalBillMean' ,
                                        'itemPastTotalBillStd' ,
                                        'itemPastTotalBillMax' ,
                                        'itemPastTotalBillMin' ,
                                        'itemPastDiscPercMean' ,
                                        'itemPastDiscPercStd' ,
                                        'itemPastDiscPercMax' ,
                                        'itemPastDiscPercMin' ,
                                        'itemPastIsCouponDiscSum' ,
                                        'itemPastIsCouponDiscMean' ,
                                        'itemPastIsMonthEndSum' ,
                                        'itemPastIsMonthEndMean' ,
                                        'itemPastIsMonthStartSum' ,
                                        'itemPastIsMonthStartMean' ,
                                        'itemPastDayOfMonthMean' ,

                                    ])

itemCount.to_csv('item_Past_info_big_imp_TRAIN_More.csv', index = False)

679 ...... 2
368 ...... 29
1489 ...... 8
523 ...... 30
205 ...... 9
793 ...... 11
761 ...... 5
1053 ...... 13
186 ...... 5
1255 ...... 30
695 ...... 8
1464 ...... 11
590 ...... 9
384 ...... 2
828 ...... 5
88 ...... 30
48 ...... 13
1351 ...... 11
1491 ...... 5
1241 ...... 2
108 ...... 9
1162 ...... 29
886 ...... 8
44 ...... 5
666 ...... 11
1050 ...... 13
375 ...... 2
712 ...... 9
1346 ...... 5
958 ...... 30
106 ...... 11
1398 ...... 8
795 ...... 5
233 ...... 2
113 ...... 29
89 ...... 13
608 ...... 9
1324 ...... 11
91 ...... 30
301 ...... 5
633 ...... 2
88 ...... 5
197 ...... 8
755 ...... 9
1058 ...... 11
1067 ...... 13
42 ...... 5
124 ...... 30
843 ...... 2
278 ...... 29
430 ...... 11
248 ...... 13
155 ...... 9
829 ...... 5
1095 ...... 8
196 ...... 2
895 ...... 30
1489 ...... 11
1194 ...... 5
569 ...... 9
1415 ...... 5
1199 ...... 11
569 ...... 29
416 ...... 2
1447 ...... 8
1152 ...... 13
1383 ...... 30
557 ...... 5
384 ...... 9
1332 ...... 11
1012 ...... 2
661 ...... 8
1334 ...... 5
77

490 ...... 30
1273 ...... 5
1124 ...... 12
582 ...... 9
1006 ...... 8
93 ...... 13
456 ...... 11
787 ...... 30
793 ...... 5
293 ...... 12
1264 ...... 9
464 ...... 5
1014 ...... 11
40 ...... 8
474 ...... 13
38 ...... 30
413 ...... 29
366 ...... 12
36 ...... 9
60 ...... 5
763 ...... 11
1384 ...... 30
450 ...... 13
1215 ...... 5
386 ...... 12
691 ...... 11
1372 ...... 9
779 ...... 8
184 ...... 29
616 ...... 5
435 ...... 30
1174 ...... 11
578 ...... 12
216 ...... 5
38 ...... 13
1479 ...... 8
1099 ...... 9
1463 ...... 11
1481 ...... 30
1167 ...... 5
1383 ...... 12
81 ...... 29
1183 ...... 13
327 ...... 9
929 ...... 11
1252 ...... 5
195 ...... 30
590 ...... 8
833 ...... 12
136 ...... 13
1023 ...... 11
1434 ...... 5
269 ...... 30
413 ...... 9
621 ...... 30
1047 ...... 5
295 ...... 12
865 ...... 29
208 ...... 11
764 ...... 8
1544 ...... 5
822 ...... 9
626 ...... 13
1293 ...... 12
699 ...... 11
1441 ...... 30
1490 ...... 5
473 ...... 8
1329 ...... 9
202 ...... 12
1157 ...... 11
1472 ...... 5
13

922 ...... 30
573 ...... 29
528 ...... 8
358 ...... 26
451 ...... 12
902 ...... 9
191 ...... 11
590 ...... 30
1125 ...... 26
1015 ...... 13
1112 ...... 8
1349 ...... 12
39 ...... 9
703 ...... 11
1 ...... 29
1320 ...... 26
840 ...... 30
1506 ...... 13
911 ...... 12
80 ...... 11
825 ...... 9
42 ...... 8
834 ...... 26
1060 ...... 30
78 ...... 11
1 ...... 12
749 ...... 9
1339 ...... 29
1004 ...... 13
938 ...... 8
294 ...... 30
506 ...... 26
767 ...... 11
1350 ...... 12
1556 ...... 9
515 ...... 11
514 ...... 8
402 ...... 26
1205 ...... 12
1073 ...... 13
1457 ...... 30
962 ...... 29
287 ...... 9
1469 ...... 11
289 ...... 8
1441 ...... 12
1363 ...... 26
1535 ...... 30
1415 ...... 11
690 ...... 9
748 ...... 13
353 ...... 8
1485 ...... 12
575 ...... 29
849 ...... 26
1121 ...... 30
918 ...... 11
44 ...... 9
74 ...... 8
1036 ...... 12
327 ...... 11
15 ...... 13
1397 ...... 26
989 ...... 30
443 ...... 9
1536 ...... 8
285 ...... 29
1547 ...... 26
612 ...... 11
1337 ...... 12
277 ...... 13
1355 ....

1465 ...... 13
1375 ...... 8
28 ...... 7
119 ...... 4
1378 ...... 28
219 ...... 7
1305 ...... 10
687 ...... 26
242 ...... 4
626 ...... 7
1098 ...... 30
1382 ...... 8
1347 ...... 13
754 ...... 4
978 ...... 10
119 ...... 7
1070 ...... 26
1463 ...... 28
1165 ...... 7
42 ...... 4
1012 ...... 30
510 ...... 10
1046 ...... 8
1320 ...... 7
440 ...... 13
78 ...... 4
1209 ...... 26
1049 ...... 7
1471 ...... 10
400 ...... 30
63 ...... 7
231 ...... 28
571 ...... 4
915 ...... 13
376 ...... 8
760 ...... 7
599 ...... 26
1083 ...... 10
1027 ...... 4
277 ...... 7
725 ...... 30
758 ...... 8
1291 ...... 7
612 ...... 10
1450 ...... 26
98 ...... 13
621 ...... 4
234 ...... 28
619 ...... 7
1223 ...... 30
675 ...... 4
8 ...... 10
412 ...... 7
979 ...... 26
1451 ...... 8
742 ...... 4
162 ...... 7
1193 ...... 13
843 ...... 30
1317 ...... 10
1140 ...... 28
895 ...... 26
1516 ...... 7
183 ...... 4
1487 ...... 30
1077 ...... 8
957 ...... 7
378 ...... 10
665 ...... 13
294 ...... 4
687 ...... 30
795 ...... 7
400 ...

189 ...... 30
171 ...... 8
820 ...... 13
283 ...... 26
1378 ...... 30
1185 ...... 8
401 ...... 13
317 ...... 26
433 ...... 30
1537 ...... 8
1035 ...... 13
782 ...... 26
1104 ...... 30
927 ...... 30
1133 ...... 13
1423 ...... 26
277 ...... 8
1029 ...... 30
436 ...... 26
405 ...... 8
752 ...... 13
992 ...... 30
442 ...... 26
1358 ...... 8
198 ...... 26
1339 ...... 13
369 ...... 30
1565 ...... 8
607 ...... 26
1521 ...... 30
550 ...... 13
783 ...... 26
757 ...... 8
685 ...... 30
478 ...... 13
1049 ...... 26
510 ...... 8
86 ...... 30
324 ...... 13
427 ...... 26
437 ...... 30
1330 ...... 8
174 ...... 13
1463 ...... 26
1017 ...... 30
342 ...... 8
1378 ...... 26
1124 ...... 13
1397 ...... 30
87 ...... 8
1457 ...... 26
1405 ...... 30
1208 ...... 13
1290 ...... 8
510 ...... 30
65 ...... 26
1550 ...... 8
87 ...... 13
1550 ...... 30
207 ...... 26
327 ...... 8
843 ...... 13
519 ...... 30
1107 ...... 26
1582 ...... 8
249 ...... 13
804 ...... 30
1140 ...... 26
1131 ...... 8
675 ...... 30
1460 ...... 

41 ...... 8
1334 ...... 8
411 ...... 13
9 ...... 13
798 ...... 8
244 ...... 13
90 ...... 8
179 ...... 8
380 ...... 13
303 ...... 13
1174 ...... 8
1287 ...... 13
194 ...... 8
1304 ...... 13
573 ...... 13
1193 ...... 8
108 ...... 8
982 ...... 13
3 ...... 8
1270 ...... 13
1064 ...... 13
577 ...... 8
702 ...... 13
1017 ...... 8
911 ...... 13
1054 ...... 8
295 ...... 13
1323 ...... 8
1101 ...... 8
1502 ...... 13
11 ...... 8
543 ...... 13
383 ...... 13
412 ...... 8
735 ...... 13
100 ...... 8
648 ...... 13
788 ...... 8
714 ...... 13
437 ...... 8
381 ...... 13
131 ...... 8
998 ...... 13
1211 ...... 8
359 ...... 13
944 ...... 13
488 ...... 8
760 ...... 8
1450 ...... 13
223 ...... 8
660 ...... 13
1506 ...... 8
1014 ...... 13
868 ...... 8
879 ...... 13
1098 ...... 13
1241 ...... 8
560 ...... 8
385 ...... 13
1394 ...... 8
1067 ...... 8
92 ...... 13
1276 ...... 8
652 ...... 13
1166 ...... 8
490 ...... 13
489 ...... 8
461 ...... 8
789 ...... 13
173 ...... 8
1008 ...... 13
14 ...... 13
675 ...... 8
1

53 ...... 8
421 ...... 13
844 ...... 8
1578 ...... 13
1315 ...... 8
1290 ...... 13
781 ...... 8
912 ...... 13
163 ...... 13
434 ...... 8
1366 ...... 13
672 ...... 8
1212 ...... 13
301 ...... 8
462 ...... 8
484 ...... 13
325 ...... 8
1084 ...... 13
154 ...... 8
851 ...... 8
1258 ...... 13
1498 ...... 13
302 ...... 8
208 ...... 8
262 ...... 13
1249 ...... 8
518 ...... 13
244 ...... 8
756 ...... 13
85 ...... 8
1497 ...... 13
1168 ...... 8
357 ...... 13
1194 ...... 8
1092 ...... 13
789 ...... 8
140 ...... 13
858 ...... 8
483 ...... 13
458 ...... 8
983 ...... 8
344 ...... 13
1103 ...... 8
1477 ...... 13
881 ...... 8
1095 ...... 13
1147 ...... 8
421 ...... 8
972 ...... 13
91 ...... 13
1302 ...... 8
1159 ...... 13
445 ...... 8
1372 ...... 13
1019 ...... 8
441 ...... 8
373 ...... 13
428 ...... 8
110 ...... 13
1197 ...... 8
744 ...... 13
745 ...... 8
708 ...... 13
466 ...... 8
1521 ...... 13
350 ...... 8
643 ...... 13
1237 ...... 8
1140 ...... 13
352 ...... 8
650 ...... 13
998 ...... 8
281 ....

1493 ...... 8
49 ...... 13
1364 ...... 8
802 ...... 13
679 ...... 8
731 ...... 8
431 ...... 13
1123 ...... 13
678 ...... 8
936 ...... 8
595 ...... 13
570 ...... 8
737 ...... 13
153 ...... 8
1403 ...... 8
1546 ...... 13
914 ...... 8
1413 ...... 13
1215 ...... 8
1431 ...... 13
1081 ...... 13
607 ...... 8
564 ...... 13
861 ...... 8
1456 ...... 8
362 ...... 13
970 ...... 8
633 ...... 13
452 ...... 8
537 ...... 13
15 ...... 8
164 ...... 13
422 ...... 8
779 ...... 13
1176 ...... 8
37 ...... 13
1485 ...... 8
805 ...... 8
1086 ...... 13
537 ...... 8
817 ...... 13
1477 ...... 8
459 ...... 13
411 ...... 8
433 ...... 13
1384 ...... 13
1203 ...... 8
501 ...... 8
761 ...... 13
1376 ...... 8
1282 ...... 8
661 ...... 13
770 ...... 8
386 ...... 13
242 ...... 8
469 ...... 13
982 ...... 8
1390 ...... 13
47 ...... 8
1434 ...... 13
542 ...... 8
505 ...... 13
1121 ...... 8
400 ...... 13
1100 ...... 8
97 ...... 13
1182 ...... 8
668 ...... 13
1091 ...... 8
631 ...... 13
1455 ...... 8
689 ...... 13
1310 .....

In [ ]:
def do_parallel_test(campIDs):
    Ans = []
    temp = test[test['campaign_id'] == campIDs]
    date = campaignData[campaignData.campaign_id == campIDs]['start_date'].values[0]
    tempTransaction = c_transaction[c_transaction.date < date]
    for custID in temp.customer_id.unique():
        print('{} ...... {}'.format(custID, campIDs))
        tempTransactionCust = tempTransaction[tempTransaction['customer_id'] == custID]
        coupnsTrain = test[(test.customer_id == custID) & (test.campaign_id == campIDs)]
        for coupID in coupnsTrain.coupon_id.unique():
            count = 0
            itemsForCoupon = coupon_item_dict[coupID]
            itemPastSellingMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].mean()
            itemPastSellingStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].std()
            itemPastSellingMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].max()
            itemPastSellingMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['selling_price'].min()
            itemPastOtherMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].mean()
            itemPastOtherStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].std()
            itemPastOtherMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].max()
            itemPastOtherMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['other_discount'].min()
            itemPastCouponMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].mean()
            itemPastCouponStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].std()
            itemPastCouponMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].max()
            itemPastCouponMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['coupon_discount'].min()
            itemPastTotal_DiscMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].mean()
            itemPastTotal_DiscStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].std()
            itemPastTotal_DiscMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].max()
            itemPastTotal_DiscMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['total_disc'].min()
            itemPastTotalBillMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].mean()
            itemPastTotalBillStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].std()
            itemPastTotalBillMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].max()
            itemPastTotalBillMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['Total_Bill'].min()
            itemPastDiscPercMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].mean()
            itemPastDiscPercStd = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].std()
            itemPastDiscPercMax = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].max()
            itemPastDiscPercMin = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['disc_perc'].min()
            itemPastIsCouponDiscSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_coupoun_disc'].sum()
            itemPastIsCouponDiscMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_coupoun_disc'].mean()
            itemPastIsMonthEndSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthEnd'].sum()
            itemPastIsMonthEndMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthEnd'].mean()
            itemPastIsMonthStartSum = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthStart'].sum()
            itemPastIsMonthStartMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['is_monthStart'].mean()
            itemPastDayOfMonthMean = tempTransactionCust[tempTransactionCust.item_id.isin(itemsForCoupon)]['dayofmonth'].mean() 
            for item in tempTransactionCust.item_id:
                if item in itemsForCoupon:
                    count+=1
            Ans.append((campIDs, custID, coupID, count, 
                       itemPastSellingMean ,
                        itemPastSellingStd ,
                        itemPastSellingMax ,
                        itemPastSellingMin ,
                        itemPastOtherMean ,
                        itemPastOtherStd ,
                        itemPastOtherMax, 
                        itemPastOtherMin , 
                        itemPastCouponMean ,
                        itemPastCouponStd , 
                        itemPastCouponMax , 
                        itemPastCouponMin , 
                        itemPastTotal_DiscMean ,
                        itemPastTotal_DiscStd , 
                        itemPastTotal_DiscMax , 
                        itemPastTotal_DiscMin , 
                        itemPastTotalBillMean , 
                        itemPastTotalBillStd ,
                        itemPastTotalBillMax , 
                        itemPastTotalBillMin , 
                        itemPastDiscPercMean , 
                        itemPastDiscPercStd , 
                        itemPastDiscPercMax , 
                        itemPastDiscPercMin , 
                        itemPastIsCouponDiscSum ,  
                        itemPastIsCouponDiscMean ,  
                        itemPastIsMonthEndSum , 
                        itemPastIsMonthEndMean ,
                        itemPastIsMonthStartSum ,  
                        itemPastIsMonthStartMean , 
                        itemPastDayOfMonthMean))
            
    return Ans

import multiprocessing

p = multiprocessing.Pool(processes=5)

Ans_test = p.map(do_parallel_test, test.campaign_id.unique())

Ans = []
for x in Ans_test:
    for i in x:
        Ans.append(i)

itemCount = pd.DataFrame(Ans, columns=['campaign_id', 'customer_id', 'coupon_id', 'prev_int_count',
                                       'itemPastSellingMean' ,
                                        'itemPastSellingStd' ,
                                        'itemPastSellingMax' ,
                                        'itemPastSellingMin' ,
                                        'itemPastOtherMean' ,
                                        'itemPastOtherStd' ,
                                        'itemPastOtherMax',
                                        'itemPastOtherMin' ,
                                        'itemPastCouponMean' ,
                                        'itemPastCouponStd' ,
                                        'itemPastCouponMax' ,
                                        'itemPastCouponMin' ,
                                        'itemPastTotal_DiscMean' ,
                                        'itemPastTotal_DiscStd' ,
                                        'itemPastTotal_DiscMax' ,
                                        'itemPastTotal_DiscMin' ,
                                        'itemPastTotalBillMean' ,
                                        'itemPastTotalBillStd' ,
                                        'itemPastTotalBillMax' ,
                                        'itemPastTotalBillMin' ,
                                        'itemPastDiscPercMean' ,
                                        'itemPastDiscPercStd' ,
                                        'itemPastDiscPercMax' ,
                                        'itemPastDiscPercMin' ,
                                        'itemPastIsCouponDiscSum' ,
                                        'itemPastIsCouponDiscMean' ,
                                        'itemPastIsMonthEndSum' ,
                                        'itemPastIsMonthEndMean' ,
                                        'itemPastIsMonthStartSum' ,
                                        'itemPastIsMonthStartMean' ,
                                        'itemPastDayOfMonthMean' ,
                                    ])

itemCount.to_csv('item_Past_info_big_imp_TEST_More.csv', index = False)

967 ...... 22
1566 ...... 20
811 ...... 17
361 ...... 25
682 ...... 18
616 ...... 17
510 ...... 22
1498 ...... 25
1303 ...... 17
565 ...... 25
744 ...... 20
1186 ...... 22
1023 ...... 18
701 ...... 17
36 ...... 25
379 ...... 20
962 ...... 25
1023 ...... 17
315 ...... 22
464 ...... 18
305 ...... 25
1442 ...... 17
1082 ...... 20
1323 ...... 22
699 ...... 25
1292 ...... 17
421 ...... 18
1538 ...... 25
963 ...... 17
704 ...... 20
45 ...... 22
1022 ...... 25
527 ...... 18
82 ...... 17
832 ...... 22
464 ...... 25
322 ...... 20
557 ...... 17
1545 ...... 18
537 ...... 25
1555 ...... 22
486 ...... 17
226 ...... 20
459 ...... 25
735 ...... 17
962 ...... 22
232 ...... 18
691 ...... 25
9 ...... 20
901 ...... 17
1134 ...... 25
569 ...... 18
338 ...... 22
680 ...... 17
483 ...... 25
453 ...... 20
859 ...... 25
884 ...... 17
145 ...... 20
1441 ...... 22
769 ...... 18
1086 ...... 25
854 ...... 17
942 ...... 18
1246 ...... 22
69 ...... 25
606 ...... 17
78 ...... 20
554 ...... 18
994 ...... 22
99 ......

891 ...... 22
1252 ...... 17
8 ...... 18
162 ...... 20
515 ...... 25
1552 ...... 17
1343 ...... 18
544 ...... 22
244 ...... 25
842 ...... 17
929 ...... 20
1553 ...... 18
1537 ...... 17
414 ...... 22
329 ...... 25
17 ...... 18
1555 ...... 20
772 ...... 17
1085 ...... 25
1061 ...... 22
1182 ...... 20
333 ...... 17
33 ...... 22
1502 ...... 18
230 ...... 25
1266 ...... 17
481 ...... 25
1400 ...... 20
989 ...... 22
339 ...... 18
517 ...... 17
948 ...... 25
737 ...... 20
1578 ...... 17
901 ...... 22
1494 ...... 18
253 ...... 20
1418 ...... 17
649 ...... 25
304 ...... 22
53 ...... 17
1168 ...... 20
847 ...... 18
569 ...... 22
510 ...... 25
1025 ...... 17
1367 ...... 22
632 ...... 20
920 ...... 25
855 ...... 18
1215 ...... 17
1327 ...... 22
920 ...... 18
58 ...... 17
1336 ...... 25
1285 ...... 20
706 ...... 22
1472 ...... 17
711 ...... 25
249 ...... 18
755 ...... 22
1005 ...... 20
467 ...... 25
990 ...... 17
1386 ...... 22
431 ...... 25
8 ...... 20
1444 ...... 17
1429 ...... 18
711 ...... 22
1

89 ...... 18
1493 ...... 22
796 ...... 23
1083 ...... 19
384 ...... 23
13 ...... 22
627 ...... 20
1004 ...... 18
907 ...... 19
766 ...... 23
965 ...... 19
666 ...... 22
948 ...... 20
1218 ...... 18
896 ...... 19
883 ...... 23
1461 ...... 22
840 ...... 19
1170 ...... 23
1534 ...... 18
1506 ...... 19
421 ...... 20
501 ...... 22
8 ...... 23
1364 ...... 19
512 ...... 18
1182 ...... 19
45 ...... 23
396 ...... 20
1015 ...... 18
1408 ...... 22
1308 ...... 19
490 ...... 23
1122 ...... 19
1393 ...... 20
755 ...... 18
228 ...... 19
1135 ...... 22
667 ...... 23
312 ...... 18
1184 ...... 19
626 ...... 22
686 ...... 20
391 ...... 18
626 ...... 23
255 ...... 19
346 ...... 18
188 ...... 19
1240 ...... 20
621 ...... 22
905 ...... 23
1223 ...... 19
306 ...... 18
756 ...... 19
1276 ...... 22
1500 ...... 23
1045 ...... 19
1017 ...... 20
616 ...... 19
275 ...... 18
911 ...... 22
962 ...... 23
1095 ...... 19
1260 ...... 19
444 ...... 20
999 ...... 23
1558 ...... 22
1358 ...... 19
704 ...... 18
1259 ...... 

1383 ...... 23
1391 ...... 16
1298 ...... 18
578 ...... 23
262 ...... 16
865 ...... 16
1430 ...... 18
444 ...... 23
247 ...... 16
784 ...... 18
353 ...... 16
426 ...... 23
606 ...... 18
900 ...... 16
1514 ...... 23
711 ...... 16
807 ...... 18
268 ...... 23
1557 ...... 18
495 ...... 16
491 ...... 16
1332 ...... 23
260 ...... 16
526 ...... 18
1305 ...... 23
537 ...... 16
1294 ...... 16
1217 ...... 18
947 ...... 23
454 ...... 16
1171 ...... 23
1182 ...... 16
1170 ...... 18
1312 ...... 16
1504 ...... 23
1103 ...... 18
1558 ...... 16
942 ...... 23
1007 ...... 16
829 ...... 18
1318 ...... 16
1158 ...... 23
1566 ...... 18
28 ...... 16
1228 ...... 23
412 ...... 16
155 ...... 23
626 ...... 16
1446 ...... 18
432 ...... 16
546 ...... 23
1249 ...... 18
1332 ...... 16
1486 ...... 23
876 ...... 16
1070 ...... 18
677 ...... 23
1428 ...... 16
546 ...... 16
924 ...... 23
267 ...... 18
407 ...... 16
303 ...... 23
1218 ...... 16
885 ...... 18
983 ...... 16
68 ...... 23
876 ...... 18
682 ...... 16
1064 ..

750 ...... 18
1317 ...... 18
906 ...... 18
465 ...... 18
181 ...... 18
1395 ...... 18
1142 ...... 18
1444 ...... 18
1235 ...... 18
1083 ...... 18
685 ...... 18
86 ...... 18
410 ...... 18
1499 ...... 18
1243 ...... 18
428 ...... 18
149 ...... 18
1195 ...... 18
864 ...... 18
168 ...... 18
711 ...... 18
1524 ...... 18
1163 ...... 18
998 ...... 18
1091 ...... 18
1368 ...... 18
1370 ...... 18
521 ...... 18
522 ...... 18
1269 ...... 18
222 ...... 18
963 ...... 18
1565 ...... 18
1464 ...... 18
324 ...... 18
900 ...... 18
735 ...... 18
672 ...... 18
1223 ...... 18
363 ...... 18
733 ...... 18
1315 ...... 18
174 ...... 18
1311 ...... 18
396 ...... 18
303 ...... 18
1184 ...... 18
314 ...... 18
833 ...... 18
1479 ...... 18
294 ...... 18
764 ...... 18
134 ...... 18
1067 ...... 18
257 ...... 18
949 ...... 18
42 ...... 18
1305 ...... 18
127 ...... 18
674 ...... 18
90 ...... 18
357 ...... 18
475 ...... 18
958 ...... 18
1025 ...... 18
806 ...... 18
572 ...... 18
1207 ...... 18
886 ...... 18
1500 ......